In [23]:
import pandas as pd

In [24]:
columns = ['SID',
           #'SEASON',
           #'NUMBER',
           #'BASIN',
           #'SUBBASIN',
           #'NAME',
           'ISO_TIME',
           #'NATURE',
           'LAT',
           'LON',
           #'WMO_WIND',
           #'WMO_PRES',
           #'TRACK_TYPE',
           #'DIST2LAND',
           #'LANDFALL',
           #'USA_WIND',
           #'USA_PRES',
           'USA_LON',
           'TOKYO_LON',
           'CMA_LON',
           'HKO_LON',
           'NEWDELHI_LON',
           'REUNION_LON',
           'BOM_LON',
           'NADI_LON',
           'WELLINGTON_LON',
           'DS824_LON',
           'TD9636_LON',
           'TD9635_LON',
           'NEUMANN_LON',
           'MLC_LON',
           #'STORM_SPEED',
           #'STORM_DIR'
          ]

In [25]:
data = pd.read_csv('./data/IBTrACS/ibtracs.since1980.list.v04r00.csv', usecols=columns, dtype='str')

In [26]:
data.head()

,SID,ISO_TIME,LAT,LON,USA_LON,TOKYO_LON,CMA_LON,HKO_LON,NEWDELHI_LON,REUNION_LON,BOM_LON,NADI_LON,WELLINGTON_LON,DS824_LON,TD9636_LON,TD9635_LON,NEUMANN_LON,MLC_LON
0,,,degrees_north,degrees_east,degrees_east,degrees_east,degrees_east,degrees_east,degrees_east,degrees_east,degrees_east,degrees_east,degrees_east,degrees_east,degrees_east,degrees_east,degrees_east,degrees_east
1,1980001S13173,1980-01-01 00:00:00,-12.5000,172.500,172.500,,,,,,,,,172.500,172.500,,172.500,
2,1980001S13173,1980-01-01 03:00:00,-12.1927,172.441,172.432,,,,,,,,,172.432,172.469,,172.432,
3,1980001S13173,1980-01-01 06:00:00,-11.9144,172.412,172.400,,,,,,,,,172.400,172.450,,172.400,
4,1980001S13173,1980-01-01 09:00:00,-11.6863,172.435,172.428,,,,,,,,,172.428,172.456,,172.428,


The following block of code removes the record in 0, in data.head()

In [27]:
ibtracs = data.drop(0, axis=0, inplace=False)
ibtracs = ibtracs.reset_index(inplace=False, drop=True)

In [28]:
ibtracs.head()

,SID,ISO_TIME,LAT,LON,USA_LON,TOKYO_LON,CMA_LON,HKO_LON,NEWDELHI_LON,REUNION_LON,BOM_LON,NADI_LON,WELLINGTON_LON,DS824_LON,TD9636_LON,TD9635_LON,NEUMANN_LON,MLC_LON
0,1980001S13173,1980-01-01 00:00:00,-12.5000,172.500,172.500,,,,,,,,,172.500,172.500,,172.500,
1,1980001S13173,1980-01-01 03:00:00,-12.1927,172.441,172.432,,,,,,,,,172.432,172.469,,172.432,
2,1980001S13173,1980-01-01 06:00:00,-11.9144,172.412,172.400,,,,,,,,,172.400,172.450,,172.400,
3,1980001S13173,1980-01-01 09:00:00,-11.6863,172.435,172.428,,,,,,,,,172.428,172.456,,172.428,
4,1980001S13173,1980-01-01 12:00:00,-11.5000,172.500,172.500,,,,,,,,,172.500,172.500,,172.500,


Let's divide the "iso_time" field into "year", "month", "day" and "hour", so that we can query for more specific values

In [29]:
ibtracs[["YEAR", "MONTH", "DAY_HOUR"]] = ibtracs["ISO_TIME"].str.split('-', expand=True)
ibtracs[["DAY", "HOUR"]] = ibtracs["DAY_HOUR"].str.split(' ', expand=True)
#btracs = ibtracs.drop(columns=["ISO_TIME"])
ibtracs = ibtracs.drop(columns=["DAY_HOUR"])
ibtracs.head()

,SID,ISO_TIME,LAT,LON,USA_LON,TOKYO_LON,CMA_LON,HKO_LON,NEWDELHI_LON,REUNION_LON,...,WELLINGTON_LON,DS824_LON,TD9636_LON,TD9635_LON,NEUMANN_LON,MLC_LON,YEAR,MONTH,DAY,HOUR
0,1980001S13173,1980-01-01 00:00:00,-12.5000,172.500,172.500,,,,,,...,,172.500,172.500,,172.500,,1980,01,01,00:00:00
1,1980001S13173,1980-01-01 03:00:00,-12.1927,172.441,172.432,,,,,,...,,172.432,172.469,,172.432,,1980,01,01,03:00:00
2,1980001S13173,1980-01-01 06:00:00,-11.9144,172.412,172.400,,,,,,...,,172.400,172.450,,172.400,,1980,01,01,06:00:00
3,1980001S13173,1980-01-01 09:00:00,-11.6863,172.435,172.428,,,,,,...,,172.428,172.456,,172.428,,1980,01,01,09:00:00
4,1980001S13173,1980-01-01 12:00:00,-11.5000,172.500,172.500,,,,,,...,,172.500,172.500,,172.500,,1980,01,01,12:00:00


Now we extract longitude and latitude of the cyclones that happened in the same ERA5 timestamp(1980-10-03T15:00:00.000000000)

Note: in the ERA5 these values are already strings, so you don't need to cast with str() in the following query

In [30]:
year = "1980"
month = "10"
day = "03"
hour = "15:00:00"
ERA5_latlon = ibtracs[(ibtracs['YEAR']==year) & (ibtracs['MONTH']==month) & (ibtracs['DAY']==day) & (ibtracs['HOUR']==hour)]
ERA5_latlon

,SID,ISO_TIME,LAT,LON,USA_LON,TOKYO_LON,CMA_LON,HKO_LON,NEWDELHI_LON,REUNION_LON,...,WELLINGTON_LON,DS824_LON,TD9636_LON,TD9635_LON,NEUMANN_LON,MLC_LON,YEAR,MONTH,DAY,HOUR
4770,1980270N13165,1980-10-03 15:00:00,36.8365,156.148,156.815,155.212,156.009,156.453,,,...,,156.815,156.406,,,,1980,10,03,15:00:00
4807,1980275N34340,1980-10-03 15:00:00,38.7285,-21.8732,-21.8732,,,,,,...,,,,,,,1980,10,03,15:00:00
4884,1980277N09157,1980-10-03 15:00:00,6.23471,153.373,153.315,153.485,153.500,153.265,,,...,,153.315,153.355,,,,1980,10,03,15:00:00


Since we searched for a specific hour, we kind of already filtered for SID, because a single cyclone cannot be in multiple places at a fixed hour.

Keep just the rows that have both lat and lon, and save just those values.

In [36]:
ERA5_latlon = ERA5_latlon[(ERA5_latlon['LAT'].notna()) & (ERA5_latlon['LON'].notna())]
lats = ERA5_latlon.LAT.values
lons = ERA5_latlon.LON.values
print(lats)
print(lons)

['36.8365' '38.7285' '6.23471']
['156.148' '-21.8732' '153.373']


In [39]:
#ibtracs[(ibtracs['YEAR']=="1997") & (ibtracs['MONTH']=="01") & (ibtracs['DAY']=="01")]

In [40]:
tmp = []
for l in ibtracs.LON:
    tmp.append(float(l))

In [41]:
max(tmp)

266.9